# This file contain full description about how to plot figure 4.6 from Jafarkhani's book
---

## Prerequisites
---  
- `Python version = 3.10`
- Latest version of numpy
- Latest version of matplotlib

<b>Note:</b>  
All Functions used in this section can be found in `useful-function` folder.

:pen: <b>Author:</b> Amirhossein Solati  
:email: <b>Contact me:</b> aamirhosseinsolati@gmail.com

In [1]:
import numpy as np
import os
# Import all our custom functions from the stbc_functions.py file
import stbc_functions as stbc

In [3]:
SNRdB = np.arange(5, 30.1, 5)
SNR = 10**(SNRdB / 10) # Make dB format to linear similar to db2pow in matlab
Nb = int(1e5)  # Number of transmitted bits (for larger numbers -> take a while to have the answer)
varh_dB = 0 # For Rayleigh channel is 1
varh = 10**(varh_dB / 10) 
RESULTS_DIR = 'results'
RESULTS_FILE = os.path.join(RESULTS_DIR, 'simulation_results_4_6.npz') # Save Numerical Results in a file for reproducibility

In [4]:
# Create results directory if it doesn't exist
os.makedirs(RESULTS_DIR, exist_ok=True)

In [ ]:
print("For N=1 and M=1 ---> SISO")
N, M = 1, 1 # Define Tx and Rx antennas as one cause it is SISO
Err1 = np.zeros(len(SNR)) # Give a empty vector of zeros which is gonna fill with counted errors
for i, snr_val in enumerate(SNR): # for all snr values we have to send Nb bit to find out BER at certain SNR
    Err = 0
    for _ in range(Nb):
        b1 = 2 * np.random.randint(0, 2) - 1 
        H = np.sqrt(varh / 2) * (np.random.randn(N, M) + 1j * np.random.randn(N, M))
        n0 = np.sqrt(1 / 2) * (np.random.randn(1, M) + 1j * np.random.randn(1, M))
        r0 = np.sqrt(snr_val / N) * b1 * H + n0
        ds_MLD = stbc.myMLD_BPSK_SISO(r0, H)
        if ds_MLD != b1:
            Err += 1
    Err1[i] = Err / Nb
    print(f"SNR (dB): {SNRdB[i]}, BER: {Err1[i]:.6f}")

For N=1 and M=1 ---> SISO
SNR (dB): 5.0, BER: 0.063870
SNR (dB): 10.0, BER: 0.022610
SNR (dB): 15.0, BER: 0.007600
SNR (dB): 20.0, BER: 0.002360
SNR (dB): 25.0, BER: 0.000860
SNR (dB): 30.0, BER: 0.000260


In [7]:
# For One iteration in above loop for SISO

print("\nGenerated random bit for bpsk:\n", b1)
print("\nGenerated random channel for bpsk:\n", H)
print("\nRecieved signal based in b1 and H:\n", r0)
print("\nDetect sent bit based on maximum likelihood:\n", ds_MLD)


Generated random bit for bpsk:
 1

Generated random channel for bpsk:
 [[0.11477056-0.55655522j]]

Recieved signal based in b1 and H:
 [[3.72870057-17.56304993j]]

Detect sent bit based on maximum likelihood:
 1


In [8]:
print("For N=2 and M=1 ---> MISO")
N, M = 2, 1 # For 2 Tx and 1 Rx antennas
Err2 = np.zeros(len(SNR)) 
for i, snr_val in enumerate(SNR):
    Err = 0
    num_loops = int(Nb / N)
    for _ in range(num_loops):
        b1 = 2 * np.random.randint(0, 2, size=(N, 1)) - 1
        C = stbc.myAlamouti(b1)
        T = C.shape[0]
        H = np.sqrt(varh / 2) * (np.random.randn(N, M) + 1j * np.random.randn(N, M))
        n0 = np.sqrt(1 / 2) * (np.random.randn(T, M) + 1j * np.random.randn(T, M))
        r = np.sqrt(snr_val / N) * (C @ H) + n0
        ds_MLD = stbc.myMLD_Alamouti_BPSK(r, H)
        Err += np.sum(ds_MLD != b1)
    Err2[i] = Err / (num_loops * N)
    print(f"SNR (dB): {SNRdB[i]}, BER: {Err2[i]:.6f}")

For N=2 and M=1 ---> MISO
SNR (dB): 5.0, BER: 0.033650
SNR (dB): 10.0, BER: 0.005450
SNR (dB): 15.0, BER: 0.000590
SNR (dB): 20.0, BER: 0.000050
SNR (dB): 25.0, BER: 0.000010
SNR (dB): 30.0, BER: 0.000000
